In [1]:
import pandas as pd
from pathlib import Path
import os
from sqlalchemy import create_engine
import numpy as np
import datetime

In [2]:
# Display options
pd.options.display.max_colwidth = 200
pd.options.display.float_format = '{:,.0f}'.format
pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.set_option('display.max_colwidth', None)

In [3]:
#  Working directory
try:
    path = Path(__file__).parent # para correr en consola
except NameError:
    path = Path('/Users/mac/Documents/IDB') #para correr en spyder
os.chdir(path)

In [4]:
# Credenciales conexion nueva BD
import modelo_predictivo.etl.credentials_newdb as credentials

In [5]:
import importlib

In [6]:
def convertir_a_minuscula(df):
    '''
    Convierte los nombres de columnas de dataframe a minuscula

    Params
    ------
    df (dataframe): dataframe 

    Returns
    ------
    dataframe con columnas en minuscula

    '''
    df.columns = map(str.lower, df.columns)
    return

    
def agregar_anio(df,variable):
    '''
    Agrega una columna con el anio extraido de una columna de dataframe de formato datetime

    Params
    ------
    df (dataframe): dataframe que contiene columna datetime de la que se extraera el anio.
    variable (string): nombre de columna que contiene la fecha desde donde se quiere extraer el anio.

    Returns
    -------
    dataframe con columna con anio

    '''
    df['anio'] = df[variable].dt.year

In [7]:
def generar_onehot_encoding(df,variable):
    '''
    Genera dataframe con variable convertida a one-hot encoding
    
    Params
    ------
    df (dataframe): dataframe al que se quiere agregar one-hot encoding y que contiene la variable a tratar
    variable (string): variable a convertir a one-hot encoding
    
    Returns
    -------
    df_rtdo (dataframe): dataframe sin variable original, reemplazada por one-hot encoding
    
    '''
    # genero onehot
    one_hot = pd.get_dummies(df[variable])
    one_hot = one_hot.add_prefix(f'{variable}_')
    
    # drop de columna
    df = df.drop(variable,axis = 1)
    
    # Join the encoded df
    df_rtdo = df.join(one_hot)
    
    return(df_rtdo)

In [8]:
# Conexion a BD
hostname = credentials.hostname
username = credentials.username
password = credentials.password
database = credentials.database
port = credentials.port

# Create an engine instance
alchemyEngine = create_engine(f'postgresql+psycopg2://{username}:{password}@{hostname}:{port}/{database}', pool_recycle=3600);

In [9]:
# Connect to PostgreSQL server
dbConnection = alchemyEngine.connect();

### Importo bases

In [10]:
# Base contratos a nivel convocatoria
contratos = pd.read_sql_table('F_CONTRATOS_NEW', con = dbConnection) 

In [11]:
contratos.shape

(202077, 35)

In [12]:
# Base convocatorias
convocatorias = pd.read_sql_table('F_CONVOCATORIAS', con = dbConnection) 

In [13]:
convocatorias.shape

(282989, 55)

In [14]:
# Base adjudicaciones 
adjudicaciones = pd.read_sql_table('F_ADJUDICACIONES', con = dbConnection) 

In [15]:
adjudicaciones.shape

(242712, 43)

In [16]:
# Base postores
postores = pd.read_sql_table('F_POSTOR', con = dbConnection) 

In [17]:
postores.shape

(1682950, 24)

In [18]:
# SUNAT
sunat = pd.read_excel('./data/sunat/SUNAT.xlsx')

In [19]:
sunat.shape

(46645, 33)

In [20]:
sunat.head()

,ruc,DDP_CIIU,DDP_DOBLE,DDP_ESTADO,DDP_FECALT,DDP_FECBAJ,DDP_IDENTI,DDP_NUMRUC,DDP_TAMANO,DDP_TPOEMP,DDP_UBIGEO,DESC_CIIU,DESC_ESTADO,ESHABIDO,DESC_TPOEMP,DDS_ASIENT,DDS_CALIFI,DDS_CIERRE,DDS_COMEXT,DDS_CONSTI,DDS_CONTAB,DDS_FICHA,DDS_INICIO,DECLARA,DDS_MOTBAJ,DESC_MOTBAJ,DESC_CIERRE,DESC_CONTAB,DESC_COMEXT,COD_PAICAP,COD_PAIORI,IND_CONLEG,DES_CONLEG
0,10000020040,70109,0,0,2008-08-13,NaT,1,10000020040,3,1,250101,ACTIVIDADES INMOBILIARIAS,ACTIVO,HABIDO,PERSONA NATURAL SIN EMP.,-,-,-,00,NaN,01,-,2008-08-13,2008-08-13,-,NaN,NaN,MANUAL,SIN ACTIVIDAD,-,-,01,PROPIO
1,10000035012,70109,0,0,2015-05-21,NaT,1,10000035012,3,1,250101,ACTIVIDADES INMOBILIARIAS,ACTIVO,HABIDO,PERSONA NATURAL SIN EMP.,-,-,-,00,NaN,00,-,2015-06-01,2015-06-01,-,NaN,NaN,SIN SISTEMA,SIN ACTIVIDAD,-,-,01,PROPIO
2,10000130121,51502,0,0,1998-06-08,NaT,1,10000130121,3,2,250101,VTA. MAY. MAQUINARIA EQUIPO Y MATER.,ACTIVO,HABIDO,PERSONA NATURAL CON EMP. UNIPERS.,-,-,99,00,NaN,01,-,1998-06-08,1998-06-08,-,NaN,PROFESION U OCUPACION INDEFINIDA,MANUAL,SIN ACTIVIDAD,-,-,-,n.d.
3,10000137397,52348,0,0,2011-03-02,NaT,1,10000137397,3,2,150136,VTA. MIN. ARTICULOS DE FERRETERIA.,ACTIVO,HABIDO,PERSONA NATURAL CON EMP. UNIPERS.,-,-,99,00,NaN,01,-,2011-03-02,2011-03-02,-,NaN,PROFESION U OCUPACION INDEFINIDA,MANUAL,SIN ACTIVIDAD,-,-,02,ALQUILADO
4,10000165838,63024,0,0,1995-06-30,NaT,1,10000165838,3,2,250101,ALMACENAMIENTO Y DEPOSITO,ACTIVO,HABIDO,PERSONA NATURAL CON EMP. UNIPERS.,-,-,99,00,NaN,01,-,1995-07-01,1995-07-01,-,NaN,PROFESION U OCUPACION INDEFINIDA,MANUAL,SIN ACTIVIDAD,-,-,-,n.d.


In [21]:
# Covid
covid = pd.read_excel('./data/data_complementaria/COVID_CONVOCATORIAS_CONTRATACIONDIRECTA.xlsx')

### Preproceso

In [22]:
# Convertir a minuscula
convertir_a_minuscula(contratos)
convertir_a_minuscula(convocatorias)
convertir_a_minuscula(adjudicaciones)
convertir_a_minuscula(postores)
convertir_a_minuscula(sunat)
convertir_a_minuscula(covid)

# Agrego año de convocatoria
agregar_anio(contratos,'fecha_suscripcion_contrato')

In [23]:
contratos.shape

(202077, 36)

In [24]:
#data types
contratos['ruc_entidad'] = contratos['ruc_entidad'].astype(int)

#### Convocatorias - base anterior

In [25]:
# Traigo convocatorias de la base de datos abiertos para completar los datos que faltan en la nueva base
# Conexion a BD
import etl.etapa_1_nueva_base.credentials_newdb as credentials

hostname = credentials.hostname_old
username = credentials.username_old
password = credentials.password_old
database = credentials.database_old
port = credentials.port_old

# Create an engine instance
alchemyEngine = create_engine(f'postgresql+psycopg2://{username}:{password}@{hostname}:{port}/{database}', pool_recycle=3600);

dbConnection  = alchemyEngine.connect();

convocatorias_old = pd.read_sql_table('convocatorias', con = dbConnection) 

In [26]:
# Genero un df con codigoconvocatorias y fechas
convocatorias_old_flat = convocatorias_old[['codigoconvocatoria','fechaintegracionbases','fechapresentacionpropuesta']].drop_duplicates()

# Join de data nueva con data de datos abiertos para traernos las fechas de integracion y presentacion de propuestas
#df
convocatorias = convocatorias.join(convocatorias_old_flat.set_index(['codigoconvocatoria']),
             on = ['codigoconvocatoria'],
             how = 'left')

#### Convocatorias

In [27]:
convocatorias_proc = convocatorias[['codigoconvocatoria','n_item',
                                    'tipoprocesoseleccion',
                                    'objetocontractual',
                                    'tipo_compra',
                                     'sector','tipoentidad',
                                    'fechaintegracionbases',
                                    'entidad_departamento', 'entidad_distrito', 'entidad_provincia',
                                    'item_departamento', 'item_provincia', 'item_distrito',
                                   'fechapresentacionpropuesta']].drop_duplicates()

In [28]:
# Join con convocatoria
contratos = contratos.join(convocatorias_proc.set_index(['codigoconvocatoria','n_item']),
                           on = ['codigoconvocatoria','num_item'],
                           how = 'inner')

#### Adjudicaciones

In [29]:
# Convierto a Int todos los itemcubso
adjudicaciones['codigogrupo'] = adjudicaciones['codigogrupo'].astype('Int64')
adjudicaciones['codigoitem'] = adjudicaciones['codigoitem'].astype('Int64')
adjudicaciones['codigofamilia'] = adjudicaciones['codigofamilia'].astype('Int64')
adjudicaciones['codigoclase'] = adjudicaciones['codigoclase'].astype('Int64')
adjudicaciones['codigocommodity'] = adjudicaciones['codigocommodity'].astype('Int64')

In [30]:
# Funcion paraagregar 00 adelante de codigoitem para que tenga 8 caracteres de largo
def agregar_ceros_codigoitem(codigoitem):
    if pd.isnull(codigoitem):
        nuevo_codigoitem = codigoitem
    else:
        if len(str(codigoitem)) == 1:
            nuevo_codigoitem = str("0000000") + str(codigoitem)
        elif len(str(codigoitem)) == 2:
            nuevo_codigoitem = str("000000") + str(codigoitem)
        elif len(str(codigoitem)) == 3:
            nuevo_codigoitem = str("00000") + str(codigoitem)
        elif len(str(codigoitem)) == 4:
            nuevo_codigoitem = str("0000") + str(codigoitem)
        elif len(str(codigoitem)) == 5:
            nuevo_codigoitem = str("000") + str(codigoitem)
        elif len(str(codigoitem)) == 6:
            nuevo_codigoitem = str("00") + str(codigoitem)
        elif len(str(codigoitem)) == 7:
            nuevo_codigoitem = str("0") + str(codigoitem)
        else:
            nuevo_codigoitem = str(codigoitem)
        nuevo_codigoitem = nuevo_codigoitem[0:8]
    return (nuevo_codigoitem)

In [31]:
# Agrego 00 adelante de codigoitem para que tenga 8 caracteres de largo
adjudicaciones['codigoitem'] = adjudicaciones.apply(lambda x : agregar_ceros_codigoitem(x['codigoitem']) 
                                      if x['codigoitem'] is not None else x['codigoitem'], axis = 1)

In [32]:
# Concatenar codigos items en adjudicaciones
adjudicaciones['grupo_familia'] = adjudicaciones['codigogrupo'].astype(str) +'-'+ adjudicaciones['codigofamilia'].astype(str)
adjudicaciones['grupo_familia_clase'] = adjudicaciones['codigogrupo'].astype(str) +'-'+ adjudicaciones['codigofamilia'].astype(str) +'-'+ adjudicaciones['codigoclase'].astype(str)
adjudicaciones['grupo_familia_clase_commodity'] = adjudicaciones['codigogrupo'].astype(str) +'-'+ adjudicaciones['codigofamilia'].astype(str) +'-'+ adjudicaciones['codigoclase'].astype(str)+'-'+ adjudicaciones['codigocommodity'].astype(str)

In [33]:
adjudicaciones['codigoitem'] = adjudicaciones['codigoitem'].where(pd.notnull(adjudicaciones['codigoitem']),None)

In [34]:
# Remove whitespaces
adjudicaciones['codigogrupo'] = adjudicaciones['codigogrupo'].astype(str).str.strip()  # or .replace as above

In [35]:
# convertir nan a none
adjudicaciones.loc[adjudicaciones['grupo_familia']== '<NA>-<NA>',['grupo_familia']] = np.nan
adjudicaciones.loc[adjudicaciones['codigogrupo'] == '<NA>',['codigogrupo']] = np.nan

In [36]:
# Agrego fechas de buena pro a contratos
adjudicaciones_proc = adjudicaciones[['codigoconvocatoria','n_item','ruc_codigo_ganador','fecha_buenapro','fecha_consentimiento_bp']].drop_duplicates()
contratos = contratos.join(adjudicaciones_proc.set_index(['codigoconvocatoria','n_item','ruc_codigo_ganador']),
                           on = ['codigoconvocatoria','num_item','ruc_codigo_ganador'],
                           how = 'left')

### Sunat

In [37]:
# data type
sunat['ruc'] = sunat['ruc'].astype(str)

In [38]:
# Agrego prefijo a nombre de columnas
sunat = sunat.add_prefix('sunat_')

In [39]:
# Reemplazo '-' por nan
cols = sunat.select_dtypes(include = [object]).columns
sunat[cols] = sunat[cols].astype(str).apply(lambda x: x.str.strip())
sunat = sunat.replace('-',np.nan)

#### Covid

In [40]:
covid['convocatoria_covid'] = 1

#### Contratos

In [41]:
contratos.shape

(201560, 51)

In [42]:
# Agrego anio de convocatoria
contratos['anio_convocatoria'] = contratos['fechaconvocatoria'].dt.year.astype(int)

In [43]:
# Agrego codigos itemcubso a contratos
adjudicaciones_proc = adjudicaciones[['codigoconvocatoria','n_item','codigogrupo','grupo_familia','codigoitem']].drop_duplicates()

contratos = contratos.join(adjudicaciones_proc.set_index(['codigoconvocatoria','n_item']), 
               on = ['codigoconvocatoria','num_item'],
              how = 'left')

In [44]:
# Agrego tipo de proveedor
tipoprov_proc = adjudicaciones[['codigoconvocatoria','ruc_codigo_ganador','tipo_proveedor']].drop_duplicates()

contratos = contratos.join(tipoprov_proc.set_index(['codigoconvocatoria','ruc_codigo_ganador']), 
               on = ['codigoconvocatoria','ruc_codigo_ganador'],
              how = 'left')

In [45]:
contratos.shape

(201560, 56)

### Universo: Filtros

In [46]:
# Lista procesos seleccionados 
tipoprocesos = ['Adjudicación Simplificada', 
                'Contratación Directa', 
                'Subasta Inversa Electrónica',
                'Licitación Pública',
                'Comparación de Precios', 
                'Concurso Público']

In [47]:
# Procesos
contratos = contratos.loc[contratos['tipoprocesoseleccion'].isin(tipoprocesos)]

In [48]:
# Objeto contractual
contratos = contratos.loc[contratos['objetocontractual'] == 'Bien']

In [49]:
# Sacamos año 2020 porque no tiene casos de sobrecostos
#contratos = contratos.loc[contratos['anio_convocatoria']!= 2020]

In [50]:
# Sacamos año 2021 porque tiene pocos sobrecostos - issue 26
#contratos = contratos.loc[contratos['anio_convocatoria']!= 2021]

In [51]:
contratos.shape

(119025, 56)

### Generacion de "Y"

#### Sobrecostos

In [52]:
# Check nulls
total_obs = contratos.shape[0]
nulls_ad = contratos["monto_adicional"].isnull().sum()
nulls_red = contratos["monto_reduccion"].isnull().sum()
nulls_pro = contratos["monto_prorroga"].isnull().sum()
nulls_com = contratos["monto_complementario"].isnull().sum()

print(f'Cantidad de observaciones = {contratos.shape[0]}')

print(f'Cantidad de nulls en monto adicional = {nulls_ad} - {round(nulls_ad/total_obs,2) * 100}%')
print(f'Cantidad de nulls en monto reduccion = {nulls_red} - {round(nulls_red/total_obs,2) * 100}%')
print(f'Cantidad de nulls en monto prorroga = {nulls_pro} - {round(nulls_pro/total_obs,2) * 100}%')
print(f'Cantidad de nulls en monto complementario = {nulls_com} - {round(nulls_com/total_obs,2) * 100}%')

Cantidad de observaciones = 119025
Cantidad de nulls en monto adicional = 11833 - 10.0%
Cantidad de nulls en monto reduccion = 11875 - 10.0%
Cantidad de nulls en monto prorroga = 11915 - 10.0%
Cantidad de nulls en monto complementario = 11898 - 10.0%


In [53]:
# Check nulls a nivel contrato
contratos_c = contratos.drop_duplicates('codigo_contrato')
total_obs = contratos_c.shape[0]
nulls_ad = contratos_c["monto_adicional"].isnull().sum()
nulls_red = contratos_c["monto_reduccion"].isnull().sum()
nulls_pro = contratos_c["monto_prorroga"].isnull().sum()
nulls_com = contratos_c["monto_complementario"].isnull().sum()

print(f'Cantidad de contratos unicos = {contratos_c.shape[0]}')

print(f'Cantidad de nulls en monto adicional = {nulls_ad} - {round(nulls_ad/total_obs,2) * 100}%')
print(f'Cantidad de nulls en monto reduccion = {nulls_red} - {round(nulls_red/total_obs,2) * 100}%')
print(f'Cantidad de nulls en monto prorroga = {nulls_pro} - {round(nulls_pro/total_obs,2) * 100}%')
print(f'Cantidad de nulls en monto complementario = {nulls_com} - {round(nulls_com/total_obs,2) * 100}%')

Cantidad de contratos unicos = 84974
Cantidad de nulls en monto adicional = 11184 - 13.0%
Cantidad de nulls en monto reduccion = 11222 - 13.0%
Cantidad de nulls en monto prorroga = 11256 - 13.0%
Cantidad de nulls en monto complementario = 11239 - 13.0%


In [54]:
# Nulls por año
x = contratos_c[['anio_convocatoria','monto_adicional','monto_reduccion','monto_prorroga','monto_complementario']]
x_na = x[x.isna().any(axis=1)]
x_na = x_na.fillna('NULL')

In [55]:
x_na.groupby('anio_convocatoria', dropna=False).count()

,monto_adicional,monto_reduccion,monto_prorroga,monto_complementario
anio_convocatoria,,,,
2021,11256,11256,11256,11256


In [56]:
# PARA VERLO SACAR FILTRO 2020!!!!!!!!!!!!!!

# Analisis de cantidad de contratos que tuvieron sobrecostos
# Cantidad de contratos con monto adicional > 0 por año
ma = contratos_c.loc[contratos_c["monto_adicional"] > 0].groupby('anio_convocatoria').count()[['index']].rename(columns = {'index':'cantidad_m_adicional_mayor_a_0'})
# Cantidad de contratos con monto reduccion > 0 por año
mr = contratos_c.loc[contratos_c["monto_reduccion"] > 0].groupby('anio_convocatoria').count()[['index']].rename(columns = {'index':'cantidad_m_reduccion_mayor_a_0'})
# Cantidad de contratos con monto prorroga > 0 por año
mp = contratos_c.loc[contratos_c["monto_prorroga"] > 0].groupby('anio_convocatoria').count()[['index']].rename(columns = {'index':'cantidad_m_prorroga_mayor_a_0'})
# Cantidad de contratos con monto complementario > 0 por año
mc = contratos_c.loc[contratos_c["monto_complementario"] > 0].groupby('anio_convocatoria').count()[['index']].rename(columns = {'index':'cantidad_m_complementario_mayor_a_0'})
z = ma.join(mr).join(mp).join(mc)
z = z.fillna(0)
z

,cantidad_m_adicional_mayor_a_0,cantidad_m_reduccion_mayor_a_0,cantidad_m_prorroga_mayor_a_0,cantidad_m_complementario_mayor_a_0
anio_convocatoria,,,,
2018,1024,311,0,682
2019,662,216,0,386
2020,9,9,0,0
2021,72,33,0,17


In [57]:
# Cantidad de contratos con monto adicional > 0 por año
cc = contratos_c.groupby('anio_convocatoria').count()[['index']].rename(columns = {'index':'cantidad_contratos'})
cc=cc.fillna(0)
cc

,cantidad_contratos
anio_convocatoria,
2018,31022
2019,32380
2020,10316
2021,11256


In [58]:
# Son de 2021
#contratos.loc[contratos["monto_reduccion"].isna()].groupby('anio').count()[['index']]
#contratos.loc[contratos["monto_adicional"].isna()].groupby('anio').count()[['index']]
#contratos.loc[contratos["monto_prorroga"].isna()].groupby('anio').count()[['index']]
#contratos.loc[contratos["monto_complementario"].isna()].groupby('anio').count()[['index']]

In [59]:
# Nan a 0
contratos['monto_adicional'].fillna(0, inplace = True)
contratos['monto_reduccion'].fillna(0, inplace = True)
contratos['monto_prorroga'].fillna(0, inplace = True)
contratos['monto_complementario'].fillna(0, inplace = True)

In [60]:
# Genero Y sobrecostos: 
#Si tuvo adicional o complementario o prorroga - sin considerar reducción - 
contratos['Y_sobrecostos'] = contratos.apply(lambda x: 1 if (x['monto_adicional']>0) or \
                                             (x['monto_prorroga']>0) or\
                                             #(x['monto_reduccion']>0) or\
                                             (x['monto_complementario']>0) else 0, axis = 1)

In [61]:
# Cantidad de 1 en Y
#Si tuvo adicional o complementario o prorroga - sin considerar reducción - 
contratos_dedup = contratos[['codigo_contrato','Y_sobrecostos']].drop_duplicates()
resumen = contratos_dedup.groupby("Y_sobrecostos",as_index = False).count()[["Y_sobrecostos",'codigo_contrato']]
resumen = resumen.append(resumen.sum(), ignore_index=True)
resumen.iloc[2,0] = 'total'
total = resumen.iloc[2,1]
resumen['%'] = resumen['codigo_contrato'] * 1.00 / (total * 1.00) * 100
resumen

,Y_sobrecostos,codigo_contrato,%
0,0,82370,97
1,1,2604,3
2,total,84974,100


In [62]:
# Genero Y sobrecostos: 
# Sumatoria de adicional + prorroga + complementario - reduccion
contratos['Y_sobrecostos_suma'] = contratos.apply(lambda x: 1 if (x['monto_adicional'] + x['monto_prorroga'] + x['monto_complementario'] - x['monto_reduccion'])
                                else 0, axis = 1)

In [63]:
# Genero Y sobrecostos: 
#contratos['Y_sobrecostos_suma'] = contratos['monto_adicional'] + contratos['monto_prorroga'] + contratos['monto_complementario'] - contratos['monto_reduccion']

In [64]:
# Genero Y sobrecostos binaria para ver cantidad de casos: 
contratos['Y_sobrecostos_suma_binaria'] = contratos.apply(lambda x: 1 if (x['Y_sobrecostos_suma']>0)
                                else 0, axis = 1)

In [65]:
# Cantidad de 1 en Y sobrecostos suma
# adicional + prorroga + complementario - reduccion
contratos_dedup = contratos[['codigo_contrato','Y_sobrecostos_suma_binaria']].drop_duplicates()
resumen = contratos_dedup.groupby("Y_sobrecostos_suma_binaria",as_index = False).count()[["Y_sobrecostos_suma_binaria",'codigo_contrato']]
resumen = resumen.append(resumen.sum(), ignore_index=True)
resumen.iloc[2,0] = 'total'
total = resumen.iloc[2,1]
resumen['%'] = resumen['codigo_contrato'] * 1.00 / (total * 1.00) * 100
resumen

,Y_sobrecostos_suma_binaria,codigo_contrato,%
0,0,81858,96
1,1,3117,4
2,total,84975,100


#### Sobretiempos

In [66]:
# Si hay contratos en 1900, cambiar a null
contratos.loc[contratos["fecha_vigencia_final"].dt.year == 1900] = np.nan
contratos.loc[contratos["fecha_vigencia_fin_actualizada"].dt.year == 1900] = np.nan

# Cantidad nulls
nulls_fin = contratos["fecha_vigencia_final"].isnull().sum()
nulls_fin_act = contratos["fecha_vigencia_fin_actualizada"].isnull().sum()

print(f'Cantidad de observaciones = {contratos.shape[0]}')

print(f'Cantidad de nulls en fecha_vigencia_final = {nulls_fin} - {round(nulls_fin/total_obs,2) * 100}%')
print(f'Cantidad de nulls en fecha_vigencia_fin_actualizada = {nulls_fin_act} - {round(nulls_fin_act/total_obs,2) * 100}%')

Cantidad de observaciones = 119025
Cantidad de nulls en fecha_vigencia_final = 0 - 0.0%
Cantidad de nulls en fecha_vigencia_fin_actualizada = 1 - 0.0%


In [67]:
# Drop observacion null
contratos = contratos[contratos['fecha_vigencia_fin_actualizada'].notna()]

In [68]:
# Genero diferencia de fechas
contratos['dif_fin_actualizada'] = contratos['fecha_vigencia_fin_actualizada'] - contratos['fecha_vigencia_final']

In [69]:
contratos[['fecha_vigencia_final','fecha_vigencia_fin_actualizada','dif_fin_actualizada']]\
.sort_values(by='dif_fin_actualizada').tail()

,fecha_vigencia_final,fecha_vigencia_fin_actualizada,dif_fin_actualizada
74838,2019-04-30 00:00:00,2020-11-20 00:00:00,570 days
110509,2020-06-01 00:00:00,2021-12-31 00:00:00,578 days
30312,2019-02-08 00:00:00,2020-10-09 00:00:00,609 days
13217,2018-05-16 10:00:00,2020-04-23 10:00:00,708 days
104391,2020-07-02 00:00:00,2022-12-31 00:00:00,912 days


In [70]:
# Genero Y sobretiempos
contratos['Y_sobretiempos'] = contratos.apply(lambda x: 1 if x['dif_fin_actualizada'].days > 0 else 0, axis = 1)

In [71]:
contratos[['fecha_vigencia_final','fecha_vigencia_fin_actualizada','dif_fin_actualizada','Y_sobretiempos']]\
.sort_values(by='dif_fin_actualizada').sample()

,fecha_vigencia_final,fecha_vigencia_fin_actualizada,dif_fin_actualizada,Y_sobretiempos
48978,2018-11-30,2018-11-30,0 days,0


In [72]:
# Cantidad de 1 en Y
contratos_dedup = contratos[['codigo_contrato','Y_sobretiempos']].drop_duplicates()
resumen = contratos_dedup.groupby("Y_sobretiempos",as_index = False).count()[["Y_sobretiempos",'codigo_contrato']]
resumen = resumen.append(resumen.sum(), ignore_index=True)
resumen.iloc[2,0] = 'total'
total = resumen.iloc[2,1]
resumen['%'] = resumen['codigo_contrato'] * 1.00 / (total * 1.00) * 100
resumen

,Y_sobretiempos,codigo_contrato,%
0,0,83533,98
1,1,1440,2
2,total,84973,100


### Elección de nivel de observaciones

Analizamos si el dataset debe ser a nivel contrato o contrato-item. Lo decidimos en base al nivel en el que esten las variables Y.

###### Check nivel de Y_sobrecostos (contrato o contrato-item)

In [73]:
# Cantidad de items distintos en cada contrato
contratos.loc[contratos["Y_sobrecostos"] == 1][['codigo_contrato','num_item']].groupby('codigo_contrato').count().sort_values(by='num_item').tail()

,num_item
codigo_contrato,
200159-00505229-01235627,16
002543-00444978-01176715,18
203086-00505229-01239372,19
200098-00444978-01205921,23
010393-00444978-01187409,27


In [74]:
# Check si el monto adicional /reduccion etc a cambia por item o es a nivel contrato
contratos.loc[contratos['codigo_contrato'] == '010393-00444978-01187409']\
[['codigo_contrato','num_item',"monto_adicional","monto_reduccion","monto_prorroga","monto_complementario","Y_sobrecostos"]].head(5)

,codigo_contrato,num_item,monto_adicional,monto_reduccion,monto_prorroga,monto_complementario,Y_sobrecostos
22562,010393-00444978-01187409,208,"9,979",0,0,0,1
72963,010393-00444978-01187409,55,"9,979",0,0,0,1
20308,010393-00444978-01187409,34,"9,979",0,0,0,1
61973,010393-00444978-01187409,145,"9,979",0,0,0,1
3425,010393-00444978-01187409,108,"9,979",0,0,0,1


In [75]:
# Check si el monto adicional /reduccion etc a cambia por item o es a nivel contrato
contratos.loc[contratos['codigo_contrato'] == '200098-00444978-01205921']\
[['codigo_contrato','num_item',"monto_adicional","monto_reduccion","monto_prorroga","monto_complementario","Y_sobrecostos"]].head()

,codigo_contrato,num_item,monto_adicional,monto_reduccion,monto_prorroga,monto_complementario,Y_sobrecostos
45089,200098-00444978-01205921,208,"5,494","2,433",0,0,1
68381,200098-00444978-01205921,186,"5,494","2,433",0,0,1
50501,200098-00444978-01205921,227,"5,494","2,433",0,0,1
12236,200098-00444978-01205921,34,"5,494","2,433",0,0,1
56550,200098-00444978-01205921,145,"5,494","2,433",0,0,1


In [76]:
# Identifico duplicados en monto adicional - 7 casos
df_ma = contratos[['codigo_contrato','monto_adicional']].drop_duplicates()
resumen = df_ma.groupby('codigo_contrato', as_index = False).count().\
sort_values(by='monto_adicional').rename(columns = {'monto_adicional':'cantidad'})
duplicados_ma = resumen.loc[resumen['cantidad']>1]['codigo_contrato'].values
len(duplicados_ma)

7

###### Check nivel de Y_sobretiempos (contrato o contrato-item)

In [77]:
# Cantidad de items distintos en cada contrato
contratos.loc[contratos["Y_sobretiempos"] == 1][['codigo_contrato','num_item']].groupby('codigo_contrato').count().sort_values(by='num_item').tail()

,num_item
codigo_contrato,
010393-00444978-01187409,27
002543-00505229-01225679,28
010214-00505229-01243009,48
203086-00505229-01239427,56
010208-00505229-01239310,61


In [78]:
# Check si fechas cambia por item o es a nivel contrato
contratos.loc[contratos['codigo_contrato'] == '010393-00444978-01187409']\
[['codigo_contrato','num_item','fecha_vigencia_final','fecha_vigencia_fin_actualizada','dif_fin_actualizada',"Y_sobretiempos"]].head(10)

,codigo_contrato,num_item,fecha_vigencia_final,fecha_vigencia_fin_actualizada,dif_fin_actualizada,Y_sobretiempos
22562,010393-00444978-01187409,208,2019-08-30,2019-09-12,13 days,1
72963,010393-00444978-01187409,55,2019-08-30,2019-09-12,13 days,1
20308,010393-00444978-01187409,34,2019-08-30,2019-09-12,13 days,1
61973,010393-00444978-01187409,145,2019-08-30,2019-09-12,13 days,1
3425,010393-00444978-01187409,108,2019-08-30,2019-09-12,13 days,1
24796,010393-00444978-01187409,184,2019-08-30,2019-09-12,13 days,1
46179,010393-00444978-01187409,26,2019-08-30,2019-09-12,13 days,1
51881,010393-00444978-01187409,136,2019-08-30,2019-09-12,13 days,1
33494,010393-00444978-01187409,82,2019-08-30,2019-09-12,13 days,1
57647,010393-00444978-01187409,92,2019-08-30,2019-09-12,13 days,1


In [79]:
# Check si fechas cambia por item o es a nivel contrato
contratos.loc[contratos['codigo_contrato'] == '002543-00505229-01225679']\
[['codigo_contrato','num_item','fecha_vigencia_final','fecha_vigencia_fin_actualizada','dif_fin_actualizada',"Y_sobretiempos"]].head(10)

,codigo_contrato,num_item,fecha_vigencia_final,fecha_vigencia_fin_actualizada,dif_fin_actualizada,Y_sobretiempos
67109,002543-00505229-01225679,84,2020-09-16,2020-11-01,46 days,1
41854,002543-00505229-01225679,331,2020-09-16,2020-11-01,46 days,1
7739,002543-00505229-01225679,270,2020-09-16,2020-11-01,46 days,1
37076,002543-00505229-01225679,234,2020-09-16,2020-11-01,46 days,1
28433,002543-00505229-01225679,301,2020-09-16,2020-11-01,46 days,1
28434,002543-00505229-01225679,376,2020-09-16,2020-11-01,46 days,1
72427,002543-00505229-01225679,250,2020-09-16,2020-11-01,46 days,1
76821,002543-00505229-01225679,154,2020-09-16,2020-11-01,46 days,1
68348,002543-00505229-01225679,33,2020-09-16,2020-11-01,46 days,1
64291,002543-00505229-01225679,265,2020-09-16,2020-11-01,46 days,1


#### Elección del código de contrato a tomar

Existen 3 columnas que hacen referencia al código de contrato, tomamos "codigo_contrato" ya que que "num_contrato" no esta normalizada y "n_cod_contrato" esta contenida en codigo_contrato.

In [80]:
contratos[['codigo_contrato','num_contrato','n_cod_contrato']].head()

,codigo_contrato,num_contrato,n_cod_contrato
1,000430-00428797-01149803,AS-001-20018-MDNP,"1,149,803"
2,010247-00427829-01148482,5 - 2018,"1,148,482"
3,001263-00427524-01148368,S/N,"1,148,368"
7,010141-00429476-01150180,165-18,"1,150,180"
8,010407-00425579-01150470,51,"1,150,470"


In [81]:
contratos['len_codigo_contrato'] = contratos['codigo_contrato'].str.len()

In [82]:
contratos.groupby('len_codigo_contrato').count()[['index']].head()

,index
len_codigo_contrato,
7,11914
24,107110


### Generacion de variables "X"

#### Caracteristicas del contrato

In [174]:
print(f'Cantidad de contratos unicos sin sacar duplicados = {contratos["codigo_contrato"].drop_duplicates().shape[0]}')

Cantidad de contratos unicos sin sacar duplicados = 84973


In [175]:
# Dataframe a nivel contratos con caracteristicas
df = contratos[['codigo_contrato',
                'Y_sobrecostos', 
                'Y_sobrecostos_suma', 
                'Y_sobretiempos',
                'codigoconvocatoria',
                'tipo_compra',
                'sector',
                'tipoentidad',
                #'anio', # es el de suscripcion de contrato
                #'anio_convocatoria',
                'tipoprocesoseleccion',
                'monto_referencial_total',
                'monto_contratado_total',
                'ruc_entidad',
                'ruc_codigo_ganador'          
               ]]
df = df.drop_duplicates()

In [176]:
# Identifico duplicados en monto referencial - ~ 450 casos
df_mr = contratos[['codigo_contrato','monto_referencial_total']].drop_duplicates()
resumen = df_mr.groupby('codigo_contrato', as_index = False).count().\
sort_values(by='monto_referencial_total').rename(columns = {'monto_referencial_total':'cantidad'})
duplicados_mr = resumen.loc[resumen['cantidad']>1]['codigo_contrato'].values

# Quito duplicados de df
df = df.loc[~df['codigo_contrato'].isin(duplicados_mr)]

In [177]:
print(f'Cantidad de contratos unicos sin montos duplicados = {df["codigo_contrato"].drop_duplicates().shape[0]}')

Cantidad de contratos unicos sin montos duplicados = 84478


In [178]:
# Onehot encoding

#variables_para_onehot = ['tipo_compra',
#                         'sector',
#                          'tipoentidad',
#                         'tipoprocesoseleccion']
#for variable in variables_para_onehot:
#    df = generar_onehot_encoding(df,variable)

#### Montos

In [179]:
# Dif % entre referencial y contrato
df['monto_referencial_vs_total'] = df['monto_contratado_total'] * 1.00/ df['monto_referencial_total'] * 1.00

In [180]:
df[['monto_referencial_vs_total','monto_contratado_total','monto_referencial_total']].head()

,monto_referencial_vs_total,monto_contratado_total,monto_referencial_total
1,1,"38,222","38,222"
2,1,"38,937","38,937"
3,0,"51,900","243,250"
7,1,"57,625","73,920"
8,1,"67,399","75,000"


In [181]:
# Cantidad de cifras de montos
df['len_monto_referencial'] = len(df['monto_referencial_total'].astype(str))

In [182]:
df['len_monto_referencial'] = df.apply(lambda x: len(str(round(x['monto_referencial_total']))), axis = 1)
df['len_monto_contratado_total'] = df.apply(lambda x: len(str(round(x['monto_contratado_total']))), axis = 1)

In [183]:
df.shape

(84479, 16)

#### Proveedores y postores

In [93]:
df_proveedores = contratos[['codigo_contrato','codigoconvocatoria']].drop_duplicates()
print(df_proveedores.shape)

(84973, 2)


In [94]:
# Cantidad de proveedores de la convocatoria
df_proveedores_cantidad = contratos[['codigoconvocatoria','ruc_codigo_ganador']]\
.groupby('codigoconvocatoria',as_index = False)['ruc_codigo_ganador'].nunique()\
.rename(columns = {'ruc_codigo_ganador': 'cantidad_proveedores'})

In [95]:
# Cantidad de postores de la convocatoria
df_postores_cantidad = postores[['codigo_convocatoria','ruc_detallepostor']]\
.groupby('codigo_convocatoria',as_index = False)['ruc_detallepostor'].nunique()\
.rename(columns = {'ruc_detallepostor': 'cantidad_postores'})
df_postores_cantidad.fillna(-9999, inplace = True)

In [96]:
# SUNAT
sunat_sel = sunat[['sunat_ruc',
                    'sunat_ddp_ciiu', 
                    'sunat_ddp_doble', 
                    'sunat_ddp_estado',
                    'sunat_ddp_identi', 
                    'sunat_ddp_tamano', 
                    'sunat_ddp_tpoemp',
                    'sunat_ddp_ubigeo', 
                    'sunat_desc_ciiu', 
                    'sunat_desc_estado',
                    'sunat_eshabido', 
                    'sunat_desc_tpoemp', 
                    'sunat_dds_cierre', 
                    'sunat_dds_comext',
                    'sunat_dds_contab', 
                    'sunat_dds_ficha',
                    'sunat_dds_motbaj', 
                    'sunat_desc_contab',
                    'sunat_desc_comext', 
                    'sunat_cod_paicap', 
                    'sunat_cod_paiori',
                    'sunat_ind_conleg', 
                    'sunat_des_conleg']]

df_sunat = contratos.join(sunat_sel.set_index('sunat_ruc'),
                         on = 'ruc_codigo_ganador',
                         how = 'left')

In [97]:
# Tipo de proveedor
df_tipo_proveedor = contratos[['codigo_contrato','tipo_proveedor']].drop_duplicates()

# Onehot encoding
#variables_para_onehot = ['tipo_proveedor']
#for variable in variables_para_onehot:
#    df_tipo_proveedor = generar_onehot_encoding(df_tipo_proveedor,variable)

In [98]:
# Proveedor con problema en el pasado
contratos['fecha_fecha_suscripcion_contrato'] = contratos['fecha_suscripcion_contrato'].dt.date #la hora genera duplicados
df_proveedor_y_pasado  = contratos[['codigo_contrato','fecha_fecha_suscripcion_contrato','ruc_codigo_ganador','Y_sobrecostos','Y_sobretiempos']].drop_duplicates()

In [99]:
# Proveedor con problema en el pasado
def identificar_proveedor_tuvo_ineficiencias_antes(df,ruc_codigo_ganador,fecha_suscripcion_contrato, tipo_ineficiencia):
    df = df.loc[(df['ruc_codigo_ganador'] == ruc_codigo_ganador) & (df['fecha_fecha_suscripcion_contrato'] < fecha_suscripcion_contrato)]
    cantidad_contratos_pasado = len(df[tipo_ineficiencia])
    cantidad_ineficiencias_pasado = len(df.loc[df[tipo_ineficiencia]==1])
    if cantidad_contratos_pasado == 0:
        proveedor_porcentaje_ineficiencias_contatos = 0
    else:
        proveedor_porcentaje_ineficiencias_contatos = (cantidad_ineficiencias_pasado * 1.00 /cantidad_contratos_pasado * 1.00) * 100
    
    return(pd.Series([cantidad_contratos_pasado,cantidad_ineficiencias_pasado,proveedor_porcentaje_ineficiencias_contatos]))

In [100]:
# Proveedor con problema en el pasado - sobrecostos
#df_proveedor_y_pasado[['proveedor_cantidad_contratos_pasado','proveedor_cantidad_sobrecostos_pasado','proveedor_porcentaje_sobrecostos_contatos']] = df_proveedor_y_pasado.apply(lambda x: identificar_proveedor_tuvo_ineficiencias_antes(df_proveedor_y_pasado,
#                                                                                                                                                      x['ruc_codigo_ganador'],
#                                                                                                                                                      x['fecha_fecha_suscripcion_contrato'],
#                                                                                                                                                      'Y_sobrecostos'), axis = 1) 

In [101]:
# Proveedor con problema en el pasado - sobretiempos
df_proveedor_y_pasado[['proveedor_cantidad_contratos_pasado','proveedor_cantidad_sobretiempos_pasado','proveedor_porcentaje_sobretiempos_contatos']] = df_proveedor_y_pasado.apply(lambda x: identificar_proveedor_tuvo_ineficiencias_antes(df_proveedor_y_pasado,
                                                                                                                                                      x['ruc_codigo_ganador'],
                                                                                                                                                      x['fecha_fecha_suscripcion_contrato'],
                                                                                                                                                      'Y_sobretiempos'), axis = 1) 

In [102]:
df_proveedor_y_pasado.drop(columns = ['fecha_fecha_suscripcion_contrato','ruc_codigo_ganador','Y_sobrecostos','Y_sobretiempos'], inplace = True)

In [103]:
# Join de sub-dataframes
print(df_proveedores.shape)

df_proveedores = df_proveedores.join(df_proveedores_cantidad.set_index('codigoconvocatoria'),
                   on = 'codigoconvocatoria', how = 'left')
df_proveedores = df_proveedores.join(df_postores_cantidad.set_index('codigo_convocatoria'),
                   on = 'codigoconvocatoria', how = 'left')
df_proveedores = df_proveedores.join(df_tipo_proveedor.set_index('codigo_contrato'),
                   on = 'codigo_contrato', how = 'left')
df_proveedores = df_proveedores.join(df_proveedor_y_pasado.set_index('codigo_contrato'),
                   on = 'codigo_contrato', how = 'left')

# drop codigoconvocatoria
df_proveedores = df_proveedores.drop('codigoconvocatoria', axis = 1)

# print
print(df_proveedores.shape)

(84973, 2)
(84973, 7)


#### Entes

In [104]:
df_ente = contratos[['codigo_contrato','ruc_entidad','fecha_fecha_suscripcion_contrato','Y_sobrecostos','Y_sobretiempos']].drop_duplicates()

In [105]:
# Ente con sobrecostos en el pasado
def identificar_ente_tuvo_ineficiencias_antes(df,ruc_entidad,fecha_suscripcion_contrato,tipo_ineficiencia):
    df = df.loc[(df['ruc_entidad'] == ruc_entidad) & (df['fecha_fecha_suscripcion_contrato'] < fecha_suscripcion_contrato)]
    cantidad_contratos_pasado = len(df[tipo_ineficiencia])
    cantidad_ineficiencia_pasado = len(df.loc[df[tipo_ineficiencia]==1])
    if cantidad_contratos_pasado == 0:
        ente_porcentaje_ineficiencia_contatos = 0
    else:
        ente_porcentaje_ineficiencia_contatos = (cantidad_ineficiencia_pasado * 1.00 /cantidad_contratos_pasado * 1.00) * 100
    
    return(pd.Series([cantidad_contratos_pasado,cantidad_ineficiencia_pasado,ente_porcentaje_ineficiencia_contatos]))


In [106]:
# Ente con sobrecosto en el pasado
#df_ente[['ente_cantidad_contratos_pasado','ente_cantidad_sobrecostos_pasado','ente_porcentaje_sobrecostos_contatos']] = df_ente.apply(lambda x: identificar_ente_tuvo_ineficiencias_antes(df_ente,
#                                                                                                                                                      x['ruc_entidad'],
#                                                                                                                                                      x['fecha_fecha_suscripcion_contrato',
#                                                                                                                                                      'Y_sobrecostos']), axis = 1) 

In [107]:
# Ente con sobretiempos en el pasado
df_ente[['ente_cantidad_contratos_pasado','ente_cantidad_sobretiempos_pasado','ente_porcentaje_sobretiempos_contatos']] = df_ente.apply(lambda x: identificar_ente_tuvo_ineficiencias_antes(df_ente,
                                                                                                                                                      x['ruc_entidad'],
                                                                                                                                                      x['fecha_fecha_suscripcion_contrato'],
                                                                                                                                                      'Y_sobretiempos'), axis = 1) 

In [108]:
df_ente.drop(columns = ['ruc_entidad','fecha_fecha_suscripcion_contrato','Y_sobrecostos','Y_sobretiempos'], inplace = True)

In [109]:
df_ente.shape

(84973, 4)

#### Entes y proveedores

In [110]:
df_ente_proveedor = contratos[['codigo_contrato','ruc_entidad','ruc_codigo_ganador','fecha_fecha_suscripcion_contrato','Y_sobrecostos','Y_sobretiempos']].drop_duplicates()

In [111]:
# Combinacion Ente-Proveedor con sobrecostos en el pasado
def identificar_ente_proveedor_tuvo_ineficiencias_antes(df,ruc_codigo_ganador,ruc_entidad,fecha_suscripcion_contrato,tipo_ineficiencia):
    df = df.loc[(df['ruc_entidad'] == ruc_entidad) & (df['ruc_codigo_ganador'] == ruc_codigo_ganador) &(df['fecha_fecha_suscripcion_contrato'] < fecha_suscripcion_contrato)]
    cantidad_contratos_pasado = len(df[tipo_ineficiencia])
    cantidad_ineficiencias_pasado = len(df.loc[df[tipo_ineficiencia]==1])
    if cantidad_contratos_pasado == 0:
        ente_proveedor_porcentaje_ineficiencia_contatos = 0
    else:
        ente_proveedor_porcentaje_ineficiencia_contatos = (cantidad_ineficiencias_pasado * 1.00 /cantidad_contratos_pasado * 1.00) * 100
    
    return(pd.Series([cantidad_contratos_pasado,cantidad_ineficiencias_pasado,ente_proveedor_porcentaje_ineficiencia_contatos]))


In [112]:
# sobrecostos
#df_ente_proveedor[['ente_proveedor_cantidad_contratos_pasado','ente_proveedor_cantidad_sobrecostos_pasado','ente_proveedor_porcentaje_sobrecostos_contatos']] = df_ente_proveedor.apply(lambda x: identificar_ente_proveedor_tuvo_ineficiencias_antes(df_ente_proveedor,
#                                                                                                                                                      x['ruc_codigo_ganador'],
#                                                                                                                                                      x['ruc_entidad'],
#                                                                                                                                                      x['fecha_fecha_suscripcion_contrato'],
#'Y_sobrecostos'), axis = 1)  

In [113]:
#sobretiempos
df_ente_proveedor[['ente_proveedor_cantidad_contratos_pasado','ente_proveedor_cantidad_ineficiencias_pasado','ente_proveedor_porcentaje_ineficiencias_contatos']] = df_ente_proveedor.apply(lambda x: identificar_ente_proveedor_tuvo_ineficiencias_antes(df_ente_proveedor,
                                                                                                                                                      x['ruc_codigo_ganador'],
                                                                                                                                                      x['ruc_entidad'],
                                                                                                                                                      x['fecha_fecha_suscripcion_contrato'],
                                                                                                                                                      'Y_sobretiempos'), axis = 1)  

In [114]:
df_ente_proveedor.drop(columns = ['ruc_entidad','ruc_codigo_ganador','fecha_fecha_suscripcion_contrato','Y_sobrecostos','Y_sobretiempos'], inplace = True)

In [115]:
df_ente_proveedor.shape

(84973, 4)

#### Items del contrato

In [165]:
df_items = contratos[['codigo_contrato']].drop_duplicates()
print(df_items.shape)

(84973, 1)


In [166]:
print(df_items.shape)

(84973, 1)


In [167]:
# montos
df_items_montos = contratos[['codigo_contrato','monto_referencial_item','monto_contratado_item']]\
.groupby('codigo_contrato').agg(['sum', 'mean','median','min','max'])
df_items_montos = df_items_montos.reset_index()
df_items_montos.columns = ['_'.join(tup).rstrip('_') for tup in df_items_montos.columns.values]
df_items_montos['monto_contratado_item_max_sobre_total'] = df_items_montos['monto_contratado_item_max'] / df_items_montos['monto_contratado_item_sum']
df_items_montos['monto_referencial_item_max_sobre_total'] = df_items_montos['monto_referencial_item_max'] / df_items_montos['monto_referencial_item_sum']

In [168]:
# cantidad
df_items_cantidad = contratos[['codigo_contrato','grupo_familia','codigogrupo','codigoitem']]\
.groupby('codigo_contrato').agg(['count'])
df_items_cantidad = df_items_cantidad.reset_index()
df_items_cantidad.columns = ['_'.join(tup).rstrip('_') for tup in df_items_cantidad.columns.values]

# dummy contrato tiene solo un item
df_items_cantidad['un_item'] =  df_items_cantidad.apply(lambda x: 1 if x['codigoitem_count'] == 1 else 0 , axis = 1)

In [169]:
# Diferencia entre sumatoria monto contratado item y monto contratado total
# Sumatoria de items
df_items_sum_items = contratos[['codigo_contrato','monto_contratado_item']]\
.groupby('codigo_contrato', as_index = False)[['monto_contratado_item']].sum()\
.rename(columns = {'monto_contratado_item': 'sum_monto_items'})

# Monto total contrato
df_items_total_contrato = contratos[['codigo_contrato','monto_contratado_total']].drop_duplicates()

# Join con df_items
df_items = df_items.join(df_items_sum_items.set_index('codigo_contrato'), on = 'codigo_contrato', how ='left')\
.join(df_items_total_contrato.set_index('codigo_contrato'), on = 'codigo_contrato', how ='left')

# Genero variable de la diferencia entre los dos montos
df_items['monto_contratado_total'] = df_items['monto_contratado_total'].astype(int)
df_items['sum_monto_items'] = df_items['sum_monto_items'].astype(int)
df_items['dif_montocontrato_sumitems'] = df_items['monto_contratado_total'] - df_items['sum_monto_items']

# Genero dummy de la variable de la diferencia entre los dos montos
df_items['dif_montocontrato_sumitems_dummy'] = df_items.apply(lambda x: 1 if x['dif_montocontrato_sumitems'] != 0 else 0, axis = 1)

In [170]:
# Onehot encoding
df_itemcubso = contratos[['codigo_contrato','codigogrupo','grupo_familia']]

variables_para_onehot = ['codigogrupo']
                         #'grupo_familia'] la saco porque genera demasiadas columnas

for variable in variables_para_onehot:
    df_itemcubso = generar_onehot_encoding(df_itemcubso,variable)


#flatteen a nivel contrato
df_itemcubso = df_itemcubso.groupby('codigo_contrato', as_index = False).sum()

#dummies
columns = df_itemcubso.drop('codigo_contrato',axis=1).columns
for col in columns:
    df_itemcubso[col].loc[df_itemcubso[col] > 1] = 1

/Users/mac/anaconda3/envs/IDB/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/Users/mac/anaconda3/envs/IDB/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/Users/mac/anaconda3/envs/IDB/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/i

In [171]:
# Join de sub-dataframes
df_items = df_items.join(df_items_montos.set_index('codigo_contrato'),
                        on = 'codigo_contrato',
                        how = 'left')

df_items = df_items.join(df_items_cantidad.set_index('codigo_contrato'),
                        on = 'codigo_contrato',
                        how = 'left')

df_items = df_items.join(df_itemcubso.set_index('codigo_contrato'),
                        on = 'codigo_contrato',
                        how = 'left')

# print
print(df_items.shape)
df_items = df_items.drop('sum_monto_items', axis = 1)
df_items = df_items.drop('monto_contratado_total', axis = 1)
df_items.head(2)

(84973, 63)


,codigo_contrato,dif_montocontrato_sumitems,dif_montocontrato_sumitems_dummy,monto_referencial_item_sum,monto_referencial_item_mean,monto_referencial_item_median,monto_referencial_item_min,monto_referencial_item_max,monto_contratado_item_sum,monto_contratado_item_mean,monto_contratado_item_median,monto_contratado_item_min,monto_contratado_item_max,monto_contratado_item_max_sobre_total,monto_referencial_item_max_sobre_total,grupo_familia_count,codigogrupo_count,codigoitem_count,un_item,codigogrupo_10,codigogrupo_11,codigogrupo_12,codigogrupo_13,codigogrupo_14,codigogrupo_15,codigogrupo_20,codigogrupo_21,codigogrupo_22,codigogrupo_23,codigogrupo_24,codigogrupo_25,codigogrupo_26,codigogrupo_27,codigogrupo_30,codigogrupo_31,codigogrupo_32,codigogrupo_39,codigogrupo_40,codigogrupo_41,codigogrupo_42,codigogrupo_43,codigogrupo_44,codigogrupo_45,codigogrupo_46,codigogrupo_47,codigogrupo_48,codigogrupo_49,codigogrupo_50,codigogrupo_51,codigogrupo_52,codigogrupo_53,codigogrupo_55,codigogrupo_56,codigogrupo_60,codigogrupo_72,codigogrupo_78,codigogrupo_80,codigogrupo_81,codigogrupo_82,codigogrupo_83,codigogrupo_95
1,000430-00428797-01149803,0,0,"38,222","38,222","38,222","38,222","38,222","38,222","38,222","38,222","38,222","38,222",1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,010247-00427829-01148482,0,0,"38,937","38,937","38,937","38,937","38,937","38,937","38,937","38,937","38,937","38,937",1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### Tiempos

In [123]:
df_tiempos = contratos[['codigoconvocatoria',
                        'codigo_contrato',
                        'fecha_suscripcion_contrato',
                        'fechapresentacionpropuesta',
                        'fecha_buenapro',
                        'fecha_consentimiento_bp',
                        'fechaconvocatoria']].groupby(['codigoconvocatoria',
                        'codigo_contrato'], as_index = False).max()
print(df_tiempos.shape)

(84973, 7)


In [124]:
# Meses 
def extraer_mes(fecha):
    return (fecha.month)

def extraer_dia(fecha):
    return (fecha.day)

In [125]:
# Mes
df_tiempos['fecha_suscripcion_contrato_mes'] = df_tiempos.apply(lambda x: extraer_mes(x['fecha_suscripcion_contrato']),
                                                               axis = 1)
df_tiempos['fechapresentacionpropuesta_mes'] = df_tiempos.apply(lambda x: extraer_mes(x['fechapresentacionpropuesta']),
                                                               axis = 1)
df_tiempos['fecha_buenapro_mes'] = df_tiempos.apply(lambda x: extraer_mes(x['fecha_buenapro']),
                                                               axis = 1)
df_tiempos['fecha_consentimiento_bp_mes'] = df_tiempos.apply(lambda x: extraer_mes(x['fecha_consentimiento_bp']),
                                                               axis = 1)
df_tiempos['fechaconvocatoria_mes'] = df_tiempos.apply(lambda x: extraer_mes(x['fechaconvocatoria']),
                                                               axis = 1)

# Dias
df_tiempos['fecha_suscripcion_contrato_dia'] = df_tiempos.apply(lambda x: extraer_dia(x['fecha_suscripcion_contrato']),
                                                               axis = 1)
df_tiempos['fechapresentacionpropuesta_dia'] = df_tiempos.apply(lambda x: extraer_dia(x['fechapresentacionpropuesta']),
                                                               axis = 1)
df_tiempos['fecha_buenapro_dia'] = df_tiempos.apply(lambda x: extraer_dia(x['fecha_buenapro']),
                                                               axis = 1)
df_tiempos['fecha_consentimiento_bp_dia'] = df_tiempos.apply(lambda x: extraer_dia(x['fecha_consentimiento_bp']),
                                                               axis = 1)
df_tiempos['fechaconvocatoria_dia'] = df_tiempos.apply(lambda x: extraer_dia(x['fechaconvocatoria']),
                                                               axis = 1)

In [126]:
# Convocatoria y suscripcion del contrato
df_tiempos['tiempos_dif_convocatoria_suscripcion'] = df_tiempos['fecha_suscripcion_contrato'] - df_tiempos['fechaconvocatoria']
# Dummy Convocatoria y suscripcion del contrato es negativo
df_tiempos['tiempos_dif_convocatoria_suscripcion_negativa'] = df_tiempos.apply(lambda x: 1 if x['tiempos_dif_convocatoria_suscripcion'] < datetime.timedelta(0)  else 0, axis = 1)
# Timedelta a integer
df_tiempos['tiempos_dif_convocatoria_suscripcion'] = df_tiempos['tiempos_dif_convocatoria_suscripcion'].dt.days

In [127]:
# Convocatoria y presentacion de propuestas
df_tiempos['tiempos_dif_convocatoria_presentacionpropuestas'] = df_tiempos['fechapresentacionpropuesta'] - df_tiempos['fechaconvocatoria']
# Dummy Convocatoria y presentacion de propuestas es negativo
df_tiempos['tiempos_dif_convocatoria_presentacionpropuestas_negativa'] = df_tiempos.apply(lambda x: 1 if x['tiempos_dif_convocatoria_presentacionpropuestas'] < datetime.timedelta(0)  else 0, axis = 1)
# Timedelta a integer
df_tiempos['tiempos_dif_convocatoria_presentacionpropuestas'] = df_tiempos['tiempos_dif_convocatoria_presentacionpropuestas'].dt.days

In [128]:
# BuenaPro y SuscripcionContrato
df_tiempos['tiempos_dif_buenapro_suscripcion'] = df_tiempos['fecha_suscripcion_contrato'] - df_tiempos['fecha_buenapro']
# Dummy Convocatoria y presentacion de propuestas es negativo
df_tiempos['tiempos_dif_buenapro_suscripcion_negativa'] = df_tiempos.apply(lambda x: 1 if x['tiempos_dif_buenapro_suscripcion'] < datetime.timedelta(0)  else 0, axis = 1)
# Timedelta a integer
df_tiempos['tiempos_dif_buenapro_suscripcion'] = df_tiempos['tiempos_dif_buenapro_suscripcion'].dt.days

In [129]:
# Consentimiento y SuscripcionContrato
df_tiempos['tiempos_dif_consentimientobuenapro_suscripcion'] = df_tiempos['fecha_suscripcion_contrato'] - df_tiempos['fecha_consentimiento_bp']
# Dummy Convocatoria y presentacion de propuestas es negativo
df_tiempos['tiempos_dif_consentimientobuenapro_suscripcion_negativa'] = df_tiempos.apply(lambda x: 1 if x['tiempos_dif_consentimientobuenapro_suscripcion'] < datetime.timedelta(0)  else 0, axis = 1)
# Timedelta a integer
df_tiempos['tiempos_dif_consentimientobuenapro_suscripcion'] = df_tiempos['tiempos_dif_consentimientobuenapro_suscripcion'].dt.days

In [130]:
# print
print(df_tiempos.shape)
df_tiempos = df_tiempos.drop(['codigoconvocatoria',
                              'fecha_suscripcion_contrato',
                             'fechapresentacionpropuesta',
                             'fecha_buenapro',
                             'fecha_consentimiento_bp',
                             'fechaconvocatoria'], axis = 1)
df_tiempos.head(2)
df_tiempos = df_tiempos.fillna(0)

(84973, 25)


#### Geografia

In [131]:
df_geo = contratos[['codigoconvocatoria','codigo_contrato','num_item','entidad_departamento','entidad_distrito','entidad_provincia',
                   'item_departamento','item_distrito','item_provincia']].drop_duplicates()
df_geo.shape

(119015, 9)

In [132]:
df_geo = df_geo.fillna(value=np.nan)

In [133]:
# comparar geografia de la entidad con geografia del item
def comparar_strings(textouno,textodos):
    if pd.isnull(textouno) or pd.isnull(textodos) :
        resultado = -1
    elif textouno == textodos:
        resultado = 0
    else:
        resultado = 1
    return(resultado)

In [134]:
# comparar geografia de la entidad con geografia del item
df_geo_comparacion = df_geo.copy()

df_geo_comparacion['geo_departamentoent_departamentoitem'] = df_geo_comparacion.apply(lambda x: comparar_strings(x['entidad_departamento'],
                                                                                       x['item_departamento']),
                                                             axis = 1)
df_geo_comparacion['geo_distritoent_distritoitem'] = df_geo_comparacion.apply(lambda x: comparar_strings(x['entidad_distrito'],
                                                                                       x['item_distrito']),
                                                             axis = 1)
df_geo_comparacion['geo_provinciaent_provinciaitem'] = df_geo_comparacion.apply(lambda x: comparar_strings(x['entidad_provincia'],
                                                                                       x['item_provincia']),
                                                             axis = 1)

In [135]:
#flatteen a nivel contrato usando max
df_geo_comparacion = df_geo_comparacion.drop(['codigoconvocatoria','num_item', 'entidad_distrito',
       'entidad_provincia', 'item_distrito', 'item_provincia'], axis = 1)

df_geo_comparacion = df_geo_comparacion.groupby('codigo_contrato', as_index = False).max()

In [136]:
df_geo_comparacion.shape

(84973, 6)

In [137]:
# onehot departamento de la entidad
#df_geo = generar_onehot_encoding(df_geo,'entidad_departamento')

In [138]:
# onehot departamento del item
#df_geo = generar_onehot_encoding(df_geo,'item_departamento')

In [139]:
#flatteen a nivel contrato usando max
df_geo = df_geo.drop(['codigoconvocatoria','num_item', 'entidad_distrito',
       'entidad_provincia', 'item_distrito', 'item_provincia'], axis = 1)

df_geo = df_geo.groupby('codigo_contrato', as_index = False).sum()

In [140]:
# join de df_geo_comparacion y df_geo
df_geo = df_geo.join(df_geo_comparacion[['codigo_contrato','geo_departamentoent_departamentoitem','geo_distritoent_distritoitem','geo_provinciaent_provinciaitem']].set_index('codigo_contrato'), 
                     on = 'codigo_contrato',
                    how = 'left')

In [141]:
#dummies
#columns = df_geo.drop(['codigo_contrato'], axis = 1).columns
columns = ['geo_departamentoent_departamentoitem','geo_distritoent_distritoitem','geo_provinciaent_provinciaitem']
for col in columns:
    df_geo[col].loc[df_geo[col] > 1] = 1

/Users/mac/anaconda3/envs/IDB/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/Users/mac/anaconda3/envs/IDB/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/Users/mac/anaconda3/envs/IDB/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/i

In [142]:
df_geo.shape

(84973, 6)

#### Covid

In [143]:
df_covid = contratos[['codigo_contrato','codigoconvocatoria']].drop_duplicates()
df_covid.shape

(84973, 2)

In [144]:
df_covid = df_covid.join(covid[['codigoconvocatoria','convocatoria_covid']].drop_duplicates().set_index('codigoconvocatoria'), 
                        on = 'codigoconvocatoria', how = 'left')

In [145]:
df_covid['convocatoria_covid'] = df_covid['convocatoria_covid'].fillna(0)

In [146]:
print(df_covid.shape)
df_covid = df_covid.drop('codigoconvocatoria', axis = 1)
df_covid.head(2)

(84973, 3)


,codigo_contrato,convocatoria_covid
1,000430-00428797-01149803,0
2,010247-00427829-01148482,0


# Dataframe

In [184]:
# X Caracteristicas
df.shape

(84479, 16)

In [185]:
# X Proveedores y postores
df = df.join(df_proveedores.set_index('codigo_contrato'),
            on = 'codigo_contrato', 
            how = 'left')
df.shape

(84479, 22)

In [186]:
# Ente
df = df.join(df_ente.set_index('codigo_contrato'),
            on = 'codigo_contrato',
            how = 'left')
df.shape

(84479, 25)

In [187]:
# Ente-Proveddor
df = df.join(df_ente_proveedor.set_index('codigo_contrato'),
            on = 'codigo_contrato',
            how = 'left')
df.shape

(84479, 28)

In [188]:
# X Items
df = df.join(df_items.set_index('codigo_contrato'),
            on = 'codigo_contrato', 
            how = 'left')
df.shape

(84479, 88)

In [189]:
# X Tiempos
df = df.join(df_tiempos.set_index('codigo_contrato'),
            on = 'codigo_contrato', 
            how = 'left')
df.shape

(84479, 106)

In [190]:
# X Covid
df = df.join(df_covid.set_index('codigo_contrato'),
            on = 'codigo_contrato', 
            how = 'left')
df.shape

(84479, 107)

In [191]:
# X Geo
df = df.join(df_geo.set_index('codigo_contrato'),
            on = 'codigo_contrato', 
            how = 'left')
df.shape

(84479, 112)

#### Analisis de dataframe

In [192]:
# Genero dataset quitando variables onehotencoding
df1 = df.loc[:, ~df.columns.str.contains('item_departamento')]
df1 = df1.loc[:, ~df1.columns.str.contains('entidad_departamento')]
df1 = df1.loc[:, ~df1.columns.str.contains('entidad_departamento')]
df1 = df1.loc[:, ~df1.columns.str.contains('codigogrupo')]

In [193]:
# Cantidad nulls en cada variable
df1.isnull().sum()

codigo_contrato                                                0
Y_sobrecostos                                                  0
Y_sobrecostos_suma                                             0
Y_sobretiempos                                                 0
codigoconvocatoria                                             0
tipo_compra                                                    0
sector                                                         0
tipoentidad                                                    0
tipoprocesoseleccion                                           0
monto_referencial_total                                        0
monto_contratado_total                                         0
ruc_entidad                                                    0
ruc_codigo_ganador                                             0
monto_referencial_vs_total                                     0
len_monto_referencial                                          0
len_monto_contratado_tota

In [194]:
df1.describe()

,Y_sobrecostos,Y_sobrecostos_suma,Y_sobretiempos,codigoconvocatoria,monto_referencial_total,monto_contratado_total,ruc_entidad,monto_referencial_vs_total,len_monto_referencial,len_monto_contratado_total,cantidad_proveedores,cantidad_postores,proveedor_cantidad_contratos_pasado,proveedor_cantidad_sobretiempos_pasado,proveedor_porcentaje_sobretiempos_contatos,ente_cantidad_contratos_pasado,ente_cantidad_sobretiempos_pasado,ente_porcentaje_sobretiempos_contatos,ente_proveedor_cantidad_contratos_pasado,ente_proveedor_cantidad_ineficiencias_pasado,ente_proveedor_porcentaje_ineficiencias_contatos,dif_montocontrato_sumitems,dif_montocontrato_sumitems_dummy,monto_referencial_item_sum,monto_referencial_item_mean,monto_referencial_item_median,monto_referencial_item_min,monto_referencial_item_max,monto_contratado_item_sum,monto_contratado_item_mean,monto_contratado_item_median,monto_contratado_item_min,monto_contratado_item_max,monto_contratado_item_max_sobre_total,monto_referencial_item_max_sobre_total,grupo_familia_count,codigoitem_count,un_item,fecha_suscripcion_contrato_mes,fechapresentacionpropuesta_mes,fecha_buenapro_mes,fecha_consentimiento_bp_mes,fechaconvocatoria_mes,fecha_suscripcion_contrato_dia,fechapresentacionpropuesta_dia,fecha_buenapro_dia,fecha_consentimiento_bp_dia,fechaconvocatoria_dia,tiempos_dif_convocatoria_suscripcion,tiempos_dif_convocatoria_suscripcion_negativa,tiempos_dif_convocatoria_presentacionpropuestas,tiempos_dif_convocatoria_presentacionpropuestas_negativa,tiempos_dif_buenapro_suscripcion,tiempos_dif_buenapro_suscripcion_negativa,tiempos_dif_consentimientobuenapro_suscripcion,tiempos_dif_consentimientobuenapro_suscripcion_negativa,convocatoria_covid,geo_departamentoent_departamentoitem,geo_distritoent_distritoitem,geo_provinciaent_provinciaitem
count,"84,479","84,479","84,479","84,479","84,479","84,479","84,479","84,479","84,479","84,479","84,479","83,927","84,479","84,479","84,479","84,479","84,479","84,479","84,479","84,479","84,479","84,479","84,479","84,479","84,479","84,479","84,479","84,479","84,479","84,479","84,479","84,479","84,479","84,362","84,479","84,479","84,479","84,479","84,479","84,479","84,479","84,479","84,479","84,479","84,479","84,479","84,479","84,479","84,479","84,479","84,479","84,479","84,479","84,479","84,479","84,479","84,479","84,479","84,479","84,479"
mean,0,0,0,"551,588","46,440,257","251,873","20,308,822,131",1,6,5,6,12,64,1,2,"1,479",16,2,42,0,1,-118,0,"8,668,304","3,950,159","2,770,907","1,866,875","8,367,559","251,992","231,913","231,136","226,212","239,650",1,1,1,1,1,7,6,6,6,7,16,15,15,15,18,60,0,13,0,17,0,24,0,0,-0,-0,-0
std,0,0,0,"80,797","120,922,088","3,127,226","177,922,268",1,1,1,12,21,166,3,6,"3,398",28,5,145,2,7,"22,275",0,"292,757,368","155,214,820","143,924,809","126,469,844","292,731,303","3,127,303","3,107,069","3,106,963","3,105,002","3,112,297",0,0,2,2,0,3,4,3,3,3,9,10,9,9,9,63,0,152,0,19,0,793,0,0,0,1,0
min,0,0,0,"424,106","11,045",0,"20,100,003,199",0,5,1,1,1,0,0,0,0,0,0,0,0,0,"-4,574,267",0,"11,045","11,045","9,445","2,562","11,045",0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,-450,0,"-43,211",0,-794,0,-457,0,0,-1,-1,-1
25%,0,0,0,"488,913","76,236","45,900","20,153,408,191",0,5,5,1,2,1,0,0,11,0,0,0,0,0,0,0,"66,410","65,000","65,000","62,756","65,641","45,863","43,648","43,470","42,209","44,910",1,1,1,1,0,5,3,4,4,4,9,6,8,7,10,23,0,1,0,9,0,3,0,0,0,0,0
50%,0,0,0,"546,909","184,874","79,790","20,199,759,672",1,6,5,1,3,6,0,0,45,0,0,0,0,0,0,0,"136,650","129,060","128,087","115,637","131,495","79,800","74,200","73,900","71,485","76,500",1,1,1,1,1,6,6,6,6,7,16,14,15,15,18,35,0,3,0,15,0,9,0,0,0,0,0
75%,0,0,0,"595,711","1,129,943","179,425","20,527,179,221",1,7,6,2,8,34,0,0,457,14,2,3,0,0,0,0,"389,074","349,789","338,575","280,777","368,332","179,615","164,992","164,500","159,771","170,000",1,1,1,1,1,9,9,9,9,11,24,23,23,23,26,68,0,12,0,23,0,14,0,0,0,0,0
max,1,1,1,"726,980","615,751,003","615,751,003","20,606,497,483",100,9,9,41,75,"1,442",23,100,"15,502",144,100,"1,314

#### Exporto dataframe

##### Sobrecostos

###### Binaria

In [195]:
# Sobrecostos binaria para Lightgbm 
#df.to_csv('modelo_predictivo/data/dataframe_v3.csv', index = False)

In [159]:
# Sobrecostos binaria con ohe
#df.to_csv('modelo_predictivo/data/dataframe_v3.csv', index = False)

##### Sobretiempos

###### Binaria

In [198]:
# Sobretiempos binaria para Lightgbm 
df.to_csv('modelo_predictivo/data/dataframe_v4_sin_ohe.csv', index = False)

In [161]:
# Sobretiempos binaria con ohe
#df.to_csv('modelo_predictivo/data/dataframe_v4.csv', index = False)